# Alerts generator

#### Liberías

In [1]:
import csv
import pandas as pd
import numpy as np

#### Dataframes

In [2]:
vigentes = pd.read_excel('../../01 Data/02 Dataset_100/semester_vig_law.xlsx').groupby('RUT')
no_vigentes = pd.read_excel('../../01 Data/02 Dataset_100/semester_novig_law.xlsx').groupby('RUT')

In [3]:
cursos = pd.read_excel('../../01 Data/02 Dataset_100/courses_vig_novig_law.xlsx').groupby('RUT')

#### Problar diccionario estudiantes

In [4]:
def num_sem(periodo_admision, periodo):
    año_admision = str(periodo_admision)[:4]
    año_periodo = str(periodo)[:4]
    sem_admision = str(periodo_admision)[-1]
    sem_periodo = str(periodo)[-1]
    dif_base = (int(año_periodo) - int(año_admision)) * 2
    if sem_admision == sem_periodo:
        dif_base += 1
    elif (sem_admision == '0') & (sem_periodo == '2'):
        dif_base += 2
    return dif_base   

In [5]:
# Vigentes
vig = {}

for estudiante_vig in vigentes:
    rut = estudiante_vig[0]
    vig[rut] = {}
    info = estudiante_vig[1]
    for i in info.index: #por semestre del estudiante
        num_semestre = num_sem(info['PERIODO ADMISIÓN'][i], info['PERIODO ACADÉMICO'][i])
        vig[rut][num_semestre] = {}
        vig[rut][num_semestre]['PERIODO ACADEMICO'] = info['PERIODO ACADÉMICO'][i]
        vig[rut][num_semestre]['SITUACION ACADEMICA'] = info['SITUACIÓN ACADÉMICA POR SEM'][i]
        vig[rut][num_semestre]['PPS'] = info['PPS'][i]
        vig[rut][num_semestre]['PPA'] = info['PPA'][i]
        vig[rut][num_semestre]['CREDITOS INSCRITOS'] = info['CRED INSCR PER'][i]
        vig[rut][num_semestre]['CREDITOS APROBADOS'] = info['CRED APROB PER'][i]
        vig[rut][num_semestre]['CREDITOS REPROBADOS'] = info['CRED REPRO PER'][i]
        vig[rut][num_semestre]['CREDITOS INSCRITOS ACUMULADOS'] = info['CRED INSCR ACUM'][i]
        vig[rut][num_semestre]['CREDITOS APROBADOS ACUMULADOS'] = info['CRED APROB ACUM'][i]
        vig[rut][num_semestre]['CREDITOS REPROBADOS ACUMULADOS'] = info['CRED REPRO ACUM'][i]      
        
for rut in vig.keys():
    num_semestre_cursado = 0
    for i in range(30):
        if i in vig[rut].keys():
            if vig[rut][i]['SITUACION ACADEMICA'] == 'INPROGRESS - En progreso':
                num_semestre_cursado += 1
                vig[rut][i]['NUM SEM CURSADO'] = num_semestre_cursado
            else:
                vig[rut][i]['NUM SEM CURSADO'] = 0 # si suspendió/anuló no tiene número de semestre cursado  
                

In [6]:
# No Vigentes
novig = {}
for estudiante_novig in no_vigentes:
    rut = estudiante_novig[0]
    novig[rut] = {}
    info = estudiante_novig[1]
    num_semestre = len(info.index)
    for i in info.index: #por semestre del estudiante
        num_semestre = num_sem(info['PERIODO ADMISIÓN'][i], info['PERIODO ACADÉMICO'][i])
        novig[rut][num_semestre] = {}
        novig[rut][num_semestre]['PERIODO ACADEMICO'] = info['PERIODO ACADÉMICO'][i]
        novig[rut][num_semestre]['SITUACION ACADEMICA'] = info['SITUACIÓN ACADÉMICA POR SEM'][i]
        novig[rut][num_semestre]['PPS'] = info['PPS'][i]
        novig[rut][num_semestre]['PPA'] = info['PPA'][i]
        novig[rut][num_semestre]['CREDITOS INSCRITOS'] = info['CRED INSCR PER'][i]
        novig[rut][num_semestre]['CREDITOS APROBADOS'] = info['CRED APROB PER'][i]
        novig[rut][num_semestre]['CREDITOS REPROBADOS'] = info['CRED REPRO PER'][i]
        novig[rut][num_semestre]['CREDITOS INSCRITOS ACUMULADOS'] = info['CRED INSCR ACUM'][i]
        novig[rut][num_semestre]['CREDITOS APROBADOS ACUMULADOS'] = info['CRED APROB ACUM'][i]
        novig[rut][num_semestre]['CREDITOS REPROBADOS ACUMULADOS'] = info['CRED REPRO ACUM'][i]      
        
for rut in novig.keys():
    num_semestre_cursado = 0
    for i in range(30):
        if i in novig[rut].keys():
            if novig[rut][i]['SITUACION ACADEMICA'] == 'INPROGRESS - En progreso':
                num_semestre_cursado += 1
                novig[rut][i]['NUM SEM CURSADO'] = num_semestre_cursado
            else:
                novig[rut][i]['NUM SEM CURSADO'] = 0 # si suspendió/anuló no tiene número de semestre cursado  
                

#### Poblar diccionario de cursos reprobados del estudiantes

In [7]:
def periodo_academico_siguiente(periodo):
    if str(periodo)[-1] == '2':
        prox_periodo = periodo + 100 - 2
    elif str(periodo)[-1] == '0':
        prox_periodo = periodo + 2
    elif str(periodo)[-1] == '4':
        prox_periodo = periodo + 100 - 4
    return prox_periodo

In [8]:
def obtener_num_sem(rut, num_sem_cursado):
    num_semestre = 0
    if rut in vig.keys():
        estudiantes = vig
    elif rut in novig.keys():
        estudiantes = novig
    for semestre in estudiantes[rut].keys():
        if estudiantes[rut][semestre]['NUM SEM CURSADO'] == num_sem_cursado:
            num_semestre = semestre
    return num_semestre

In [9]:
def obtener_num_sem_cursados(rut, periodo):
    num_sem_cursados = 0
    if str(periodo)[-1] == '4':
        periodo -= 2
    if rut in vig.keys():
        estudiantes = vig
    elif rut in novig.keys():
        estudiantes = novig
    for semestre in estudiantes[rut].keys():
        if estudiantes[rut][semestre]['PERIODO ACADEMICO'] == periodo:
            num_sem_cursados = estudiantes[rut][semestre]['NUM SEM CURSADO']
            break
    return num_sem_cursados

In [10]:
estudiantes_cursos_rep = {}
for estudiante in cursos:
    rut = estudiante[0]
    estudiantes_cursos_rep[rut] = {}
    for periodo in estudiante[1].groupby('PERIODO'):
        for i in periodo[1].index:
            if not (periodo[1]['CALIF'][i] in ['A', 'R', 'P', 'C']) and not(pd.isnull(periodo[1]['CALIF'][i])):
                if float(periodo[1]['CALIF'][i]) < 4.0:
                    sigla_curso = periodo[1]['MATERIA'][i] + str(periodo[1]['CURSO'][i])
                    if not (sigla_curso in estudiantes_cursos_rep[rut].keys()):
                        estudiantes_cursos_rep[rut][sigla_curso] = 0
                    estudiantes_cursos_rep[rut][sigla_curso] += 1
                    num_sem_cursados = obtener_num_sem_cursados(rut, periodo[0])
                    # ALERTA 28f
                    if estudiantes_cursos_rep[rut][sigla_curso] >= 3: # Si lo reprobó 3 veces o más
                        periodo_alerta = periodo_academico_siguiente(periodo[0])
                        if not ('alerta_28f' in estudiantes_cursos_rep[rut].keys()):
                            estudiantes_cursos_rep[rut]['alerta_28f'] = []
                        if not (periodo_alerta, obtener_num_sem(rut, num_sem_cursados)) in estudiantes_cursos_rep[rut]['alerta_28f']:
                            estudiantes_cursos_rep[rut]['alerta_28f'].append((periodo_alerta, obtener_num_sem(rut, num_sem_cursados))) #periodo en que recibe la alerta y número de periodos en la carrera
                    
                    # ALERTA 28g
                    if estudiantes_cursos_rep[rut][sigla_curso] >= 2: # Si lo reprobó dos veces o más
                        if num_sem_cursados > 2: # Si está en un semestre posterior al segundo
                            # Si el curso es específico de UA
                            if sigla_curso in ['DER001C', 'DER001F', 'DER001L', 'DER001P', 'DER001R', 'DER002C', 'DER002F', 'DER002P', 'DER002R', 'DER003C', 'DER003F', 'DER003L', 'DER004C', 'DER004F', 'DER005F', 'DER005R', 'DER006F', 'DER008C', 'DER009F']: 
                                periodo_alerta = periodo_academico_siguiente(periodo[0])
                                if not ('alerta_28g' in estudiantes_cursos_rep[rut].keys()):
                                    estudiantes_cursos_rep[rut]['alerta_28g'] = []
                                if not (periodo_alerta, obtener_num_sem(rut, num_sem_cursados)) in estudiantes_cursos_rep[rut]['alerta_28g']:
                                    estudiantes_cursos_rep[rut]['alerta_28g'].append((periodo_alerta, obtener_num_sem(rut, num_sem_cursados))) #periodo en que recibe la alerta y número de periodos en la carrera           

#### Simulación alertas

In [11]:
alertas = {}

# Vigentes
estudiantes = vig

# 28a
for rut in estudiantes.keys():
    primer_semestre = obtener_num_sem(rut, 1)
    if primer_semestre != 0:
        if estudiantes[rut][primer_semestre]['PPA'] < 4.0:
            prox_periodo = periodo_academico_siguiente(estudiantes[rut][primer_semestre]['PERIODO ACADEMICO'])
            if not (prox_periodo in alertas.keys()):
                alertas[prox_periodo] = []
            alertas[prox_periodo].append(['28a', rut, primer_semestre]) #[tipo alerta, rut, num periodos en la carrera]

# 28b
for rut in estudiantes.keys():
    primer_semestre = obtener_num_sem(rut, 1)
    if primer_semestre != 0:
        porcentaje_creditos_aprobados = estudiantes[rut][primer_semestre]['CREDITOS APROBADOS']/estudiantes[rut][primer_semestre]['CREDITOS INSCRITOS']
        if porcentaje_creditos_aprobados < 0.5:
            prox_periodo = periodo_academico_siguiente(estudiantes[rut][primer_semestre]['PERIODO ACADEMICO'])
            if not (prox_periodo in alertas.keys()):
                alertas[prox_periodo] = []
            alertas[prox_periodo].append(['28b', rut, primer_semestre]) #[tipo alerta, rut, num periodos en la carrera]

# 28c
for rut in estudiantes.keys():
    segundo_semestre = obtener_num_sem(rut, 2)
    if segundo_semestre != 0:
        if estudiantes[rut][segundo_semestre]['PPA'] < 4.0:
            prox_periodo = periodo_academico_siguiente(estudiantes[rut][segundo_semestre]['PERIODO ACADEMICO'])
            if not (prox_periodo in alertas.keys()):
                alertas[prox_periodo] = []
            alertas[prox_periodo].append(['28c', rut, segundo_semestre]) #[tipo alerta, rut, num periodos en la carrera]
# 28d
for rut in estudiantes.keys():
    segundo_semestre = obtener_num_sem(rut, 2)
    if segundo_semestre != 0:
        creditos_aprobados_acumulados= estudiantes[rut][segundo_semestre]['CREDITOS APROBADOS ACUMULADOS']
        creditos_inscritos_acumulados= estudiantes[rut][segundo_semestre]['CREDITOS INSCRITOS ACUMULADOS']
        porcentaje_creditos_aprobados = creditos_aprobados_acumulados/ creditos_inscritos_acumulados
        if estudiantes[rut][segundo_semestre]['PPA'] < 4.5 and porcentaje_creditos_aprobados < 0.4:
            prox_periodo = periodo_academico_siguiente(estudiantes[rut][segundo_semestre]['PERIODO ACADEMICO'])
            if not (prox_periodo in alertas.keys()):
                alertas[prox_periodo] = []
            alertas[prox_periodo].append(['28d', rut, segundo_semestre]) #[tipo alerta, rut, num periodos en la carrera]
                
# 28e
for rut in estudiantes.keys():
    for i in range(3,40):
        semestre = obtener_num_sem(rut, i)
        if semestre != 0:
            if estudiantes[rut][semestre]['PPA'] < 4.0:
                prox_periodo = periodo_academico_siguiente(estudiantes[rut][semestre]['PERIODO ACADEMICO'])
                if not (prox_periodo in alertas.keys()):
                    alertas[prox_periodo] = []
                alertas[prox_periodo].append(['28e', rut, semestre]) #[tipo alerta, rut, num periodos en la carrera]

In [12]:
# No Vigentes
estudiantes = novig

# 28a
for rut in estudiantes.keys():
    primer_semestre = obtener_num_sem(rut, 1)
    if primer_semestre != 0:
        if estudiantes[rut][primer_semestre]['PPA'] < 4.0:
            prox_periodo = periodo_academico_siguiente(estudiantes[rut][primer_semestre]['PERIODO ACADEMICO'])
            if not (prox_periodo in alertas.keys()):
                alertas[prox_periodo] = []
            alertas[prox_periodo].append(['28a', rut, primer_semestre]) #[tipo alerta, rut, num periodos en la carrera]

# 28b
for rut in estudiantes.keys():
    primer_semestre = obtener_num_sem(rut, 1)
    if primer_semestre != 0:
        porcentaje_creditos_aprobados = estudiantes[rut][primer_semestre]['CREDITOS APROBADOS']/estudiantes[rut][primer_semestre]['CREDITOS INSCRITOS']
        if porcentaje_creditos_aprobados < 0.5:
            prox_periodo = periodo_academico_siguiente(estudiantes[rut][primer_semestre]['PERIODO ACADEMICO'])
            if not (prox_periodo in alertas.keys()):
                alertas[prox_periodo] = []
            alertas[prox_periodo].append(['28b', rut, primer_semestre]) #[tipo alerta, rut, num periodos en la carrera]

# 28c
for rut in estudiantes.keys():
    segundo_semestre = obtener_num_sem(rut, 2)
    if segundo_semestre != 0:
        if estudiantes[rut][segundo_semestre]['PPA'] < 4.0:
            prox_periodo = periodo_academico_siguiente(estudiantes[rut][segundo_semestre]['PERIODO ACADEMICO'])
            if not (prox_periodo in alertas.keys()):
                alertas[prox_periodo] = []
            alertas[prox_periodo].append(['28c', rut, segundo_semestre]) #[tipo alerta, rut, num periodos en la carrera]
# 28d
for rut in estudiantes.keys():
    segundo_semestre = obtener_num_sem(rut, 2)
    if segundo_semestre != 0:
        creditos_aprobados_acumulados= estudiantes[rut][segundo_semestre]['CREDITOS APROBADOS ACUMULADOS']
        creditos_inscritos_acumulados= estudiantes[rut][segundo_semestre]['CREDITOS INSCRITOS ACUMULADOS']
        porcentaje_creditos_aprobados = creditos_aprobados_acumulados/ creditos_inscritos_acumulados
        if estudiantes[rut][segundo_semestre]['PPA'] < 4.5 and porcentaje_creditos_aprobados < 0.4:
            prox_periodo = periodo_academico_siguiente(estudiantes[rut][segundo_semestre]['PERIODO ACADEMICO'])
            if not (prox_periodo in alertas.keys()):
                alertas[prox_periodo] = []
            alertas[prox_periodo].append(['28d', rut, segundo_semestre]) #[tipo alerta, rut, num periodos en la carrera]
                
# 28e
for rut in estudiantes.keys():
    for i in range(3,40):
        semestre = obtener_num_sem(rut, i)
        if semestre != 0:
            if estudiantes[rut][semestre]['PPA'] < 4.0:
                prox_periodo = periodo_academico_siguiente(estudiantes[rut][semestre]['PERIODO ACADEMICO'])
                if not (prox_periodo in alertas.keys()):
                    alertas[prox_periodo] = []
                alertas[prox_periodo].append(['28e', rut, semestre]) #[tipo alerta, rut, num periodos en la carrera]

In [13]:
# Vigentes y No Vigentes
# 28f
for rut in estudiantes_cursos_rep.keys():
    if ('alerta_28f' in estudiantes_cursos_rep[rut].keys()):
        for alerta_28f in estudiantes_cursos_rep[rut]['alerta_28f']:
            if not (alerta_28f[0] in alertas.keys()):
                    alertas[alerta_28f[0]] = []
            alertas[alerta_28f[0]].append(['28f', rut, alerta_28f[1]])

# 28g
    if ('alerta_28g' in estudiantes_cursos_rep[rut].keys()):
        for alerta_28g in estudiantes_cursos_rep[rut]['alerta_28g']:
            if not (alerta_28g[0] in alertas.keys()):
                    alertas[alerta_28g[0]] = []
            alertas[alerta_28g[0]].append(['28g', rut, alerta_28g[1]])    

alertas_d = alertas
%store alertas_d

Stored 'alertas_d' (dict)
